In [8]:
import os
import shutil

src_root = r"D:\Code and stuff\heck-a-ton\archive\images\images"                # original dataset root
dst_root = r"D:\Code and stuff\heck-a-ton\archive\dataset_flattened"     # where flattened dataset will go
os.makedirs(dst_root, exist_ok=True)

for class_name in os.listdir(src_root):
    class_path = os.path.join(src_root, class_name)
    if not os.path.isdir(class_path):
        continue
    
    # Create destination folder for this class
    dst_class_path = os.path.join(dst_root, class_name)
    os.makedirs(dst_class_path, exist_ok=True)
    
    # Go through "default" and "real_world"
    for subfolder in ["default", "real_world"]:
        sub_path = os.path.join(class_path, subfolder)
        if not os.path.isdir(sub_path):
            continue
        
        for i, fname in enumerate(os.listdir(sub_path)):
            src_file = os.path.join(sub_path, fname)
            
            # rename with prefix so no conflicts
            new_name = f"{subfolder}_{i}_{fname}"
            dst_file = os.path.join(dst_class_path, new_name)
            
            shutil.copy(src_file, dst_file)

print("Flattening + renaming done! Check 'dataset_flattened/'")


Flattening + renaming done! Check 'dataset_flattened/'


In [6]:
import os

src_root = r"D:\Code and stuff\heck-a-ton\archive\images"   # adjust if needed
for class_name in os.listdir(src_root):
    class_path = os.path.join(src_root, class_name)
    if not os.path.isdir(class_path):
        continue
    
    for subfolder in ["default", "real_world"]:
        sub_path = os.path.join(class_path, subfolder)
        if os.path.isdir(sub_path):
            files = [f for f in os.listdir(sub_path) if f.lower().endswith(('.png','.jpg','.jpeg'))]
            print(class_name, subfolder, "->", len(files), "images")


In [7]:
import os

print("Listing contents of 'images/':")
print(os.listdir(r"D:\Code and stuff\heck-a-ton\archive\images"))


Listing contents of 'images/':
['images']


In [9]:
import random

src_root = r"D:\Code and stuff\heck-a-ton\archive\dataset_flattened"
dst_root = r"D:\Code and stuff\heck-a-ton\archive\dataset_split"

for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(dst_root, split), exist_ok=True)

split_ratio = {"train": 0.7, "val": 0.15, "test": 0.15}

for class_name in os.listdir(src_root):
    class_path = os.path.join(src_root, class_name)
    if not os.path.isdir(class_path):
        continue
    
    files = os.listdir(class_path)
    random.shuffle(files)
    
    n_total = len(files)
    n_train = int(n_total * split_ratio["train"])
    n_val   = int(n_total * split_ratio["val"])
    
    split_files = {
        "train": files[:n_train],
        "val": files[n_train:n_train+n_val],
        "test": files[n_train+n_val:]
    }
    
    for split, filenames in split_files.items():
        dst_class_path = os.path.join(dst_root, split, class_name)
        os.makedirs(dst_class_path, exist_ok=True)
        
        for fname in filenames:
            src_file = os.path.join(class_path, fname)
            dst_file = os.path.join(dst_class_path, fname)
            shutil.copy(src_file, dst_file)

print("Train/Val/Test split done! Check 'dataset_split/'")


Train/Val/Test split done! Check 'dataset_split/'


In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

# Paths
data_dir = r"D:\Code and stuff\heck-a-ton\archive\dataset_flattened"   # <-- your renamed + flattened dataset


In [6]:
# Standard transforms for training vs validation
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [10]:
# Full dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transforms)

# Train/val/test split (70/20/10)
train_size = int(0.7 * len(full_dataset))
val_size   = int(0.2 * len(full_dataset))
test_size  = len(full_dataset) - train_size - val_size

train_ds, val_ds, test_ds = random_split(full_dataset, [train_size, val_size, test_size])

# Apply val transforms to val & test
val_ds.dataset.transform = val_transforms
test_ds.dataset.transform = val_transforms

# DataLoaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=2)

print(f"Train: {len(train_ds)} | Val: {len(val_ds)} | Test: {len(test_ds)}")


Train: 10500 | Val: 3000 | Test: 1500


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

num_classes = len(full_dataset.classes)

# Load pretrained model
model = models.resnet50(pretrained=True)

# Freeze backbone
for param in model.parameters():
    param.requires_grad = False

# Replace classifier head
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, num_classes)
)

model = model.to(device)


Using: cuda


d:\Code and stuff\Neural Networks\PyTorch_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Code and stuff\Neural Networks\PyTorch_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\sharm/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [13]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    train_acc, val_acc = [], []

    for epoch in range(epochs):
        model.train()
        correct, total, running_loss = 0, 0, 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_train_acc = correct / total
        train_acc.append(epoch_train_acc)

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        epoch_val_acc = correct / total
        val_acc.append(epoch_val_acc)

        print(f"Epoch {epoch+1}/{epochs} - Train Acc: {epoch_train_acc:.3f} - Val Acc: {epoch_val_acc:.3f}")

    return train_acc, val_acc

train_acc, val_acc = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)


Epoch 1/5 - Train Acc: 0.561 - Val Acc: 0.711
Epoch 2/5 - Train Acc: 0.703 - Val Acc: 0.736
Epoch 3/5 - Train Acc: 0.733 - Val Acc: 0.758
Epoch 4/5 - Train Acc: 0.753 - Val Acc: 0.759
Epoch 5/5 - Train Acc: 0.768 - Val Acc: 0.775


In [14]:
torch.save(model.state_dict(), "waste_classifier.pth")
print("Model saved!")


Model saved!


In [19]:
from PIL import Image

def predict_image(image_path, model, class_names):
    model.eval()
    img = Image.open(image_path).convert("RGB")
    transform = val_transforms
    img_t = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_t)
        _, preds = torch.max(outputs, 1)
    
    return class_names[preds[0].item()]

# Example
print(predict_image(r"D:\Code and stuff\heck-a-ton\archive\dataset_flattened\plastic_water_bottles\default_0_image_1.png", model, full_dataset.classes))


plastic_water_bottles


In [29]:
broad_map = {
    # Plastic
    "plastic_water_bottles": "Plastic",
    "plastic_soda_bottles": "Plastic",
    "plastic_detergent_bottles": "Plastic",
    "plastic_shopping_bags": "Plastic",
    "plastic_trash_bags": "Plastic",
    "plastic_food_containers": "Plastic",
    "disposable_plastic_cutlery": "Plastic",
    "plastic_straws": "Plastic",
    "plastic_cup_lids": "Plastic",
    "paper_cups": 'Paper',

    # Paper & Cardboard
    "newspaper": "Paper",
    "office_paper": "Paper",
    "magazines": "Paper",
    "cardboard_boxes": "Cardboard",
    "cardboard_packaging": "Cardboard",

    # Glass
    "glass_beverage_bottles": "Glass",
    "glass_food_jars": "Glass",
    "glass_cosmetic_containers": "Glass",

    # Metal
    "aluminum_soda_cans": "Metal",
    "aluminum_food_cans": "Metal",
    "steel_food_cans": "Metal",
    "aerosol_cans": "Metal",

    # Organic
    "food_waste": "Organic",
    "eggshells": "Organic",
    "coffee_grounds": "Organic",
    "tea_bags": "Organic",

    # Textiles
    "clothing": "Textiles",
    "shoes": "Textiles"
}

# Usage
pred = predict_image(r"D:\Code and stuff\heck-a-ton\archive\dataset_flattened\newspaper\default_6_Image_104.png", model, full_dataset.classes)
print("Fine-grained:", pred)
print("Broad:", broad_map.get(pred, "Unknown"))


Fine-grained: newspaper
Broad: Paper
